The classification is performed by following:

 1. The function's energy and power are computed with definite integral with bounds from  $0$ to $10^2$, $10^3$, $10^4$, ... : 
     $$
     E_1 = \int_0 ^ {10^2} f^2(x) dx \\
     E_2 = \int_0 ^ {10^3} f^2(x) dx \\
     E_3 = \int_0 ^ {10^4} f^2(x) dx \\
     \vdots
     $$
     
     At the same time power is computed:
     $$
     P_1 = \frac{1}{10^2}\int_0 ^ {10^2} f^2(x) dx \\
     P_2 = \frac{1}{10^3}\int_0 ^ {10^3} f^2(x) dx \\
     P_3 = \frac{1}{10^4}\int_0 ^ {10^4} f^2(x) dx \\
     \vdots
     $$
     
     In the program, integrals are evaluvated using Trapezoidal rule (https://en.wikipedia.org/wiki/Trapezoidal_rule#Uniform_grid). Also a separate evaluvator function is used in summation. The evaluvator checks for overflows and zero divisions:
     
     For energy:
     $$e(x) = 
       \begin{cases}
        \infty             &, \text{if } f^2(x)dx \text{ overflows } \\
        \infty             &, \text{if } f(x) \text{ involves zero division} \\
        f(x) ^ 2 dx &, \text{otherwise}
       \end{cases}
     $$
     
     For power:
     $$p(x) = 
       \begin{cases}
        \infty             &, \text{if } \dfrac{f^2(x)dx}{duration}\ \text{ overflows or } \\
        \infty             &, \text{if } f(x) \text{ involves zero division} \\
        \dfrac{f^2(x)dx}{duration}&, \text{otherwise}
       \end{cases}
     $$
     
     Integration is done by:
     $$
     E = \frac{e(start) + e(end)}{2} + \sum_{start+dx} ^ {end-dx} e(x) \\
     P = \frac{p(start) + p(end)}{2} + \sum_{start+dx} ^ {end-dx} p(x)
     $$
 2. Check if they tend to increase, ie $E_1 < E_2 < E_3 \cdots$ .If they do so, Its not Energy signal (Hence find poewr)
 4. Check if they tend to increase, ie $P_1 < P_2 < P_3 \cdots$ .If they do so, Its not Power signal either.
 5. Check for oscillatory behaviour
    

In [22]:
from math import sqrt, sin, e, inf
def f1(x): return sin(x) * e**x
def f2(x): return e**x
def f3(x): return 1 if x >= 0 else 0
def f4(x): return x
def f5(x): return 1/x
def f6(x): return sqrt(x)

In [19]:
# Computes energy and power
def compute_E_P(f, start, end, dx=0.01):
    duration  = end-start
    energy_overflown = False
    power_overflown = False
    
    # Evaluvators for energy and power. It also check overlfow and zero division
    
    def eval_energy(x):
        k = 0
        try:
            k = f(x) ** 2 * dx
        except OverflowError:
            k = inf
            energy_overflown = True
        except ZeroDivisionError:
            k = inf
        return k
    
    def eval_power(x):
        k = 0
        try:
            k = f(x) ** 2 * dx / duration
        except OverflowError:
            k = inf
            power_overflown = True
        except ZeroDivisionError:
            k = inf
        return k
    
    
    # using trapezoidal rule to integerate
    
    # add initial parts
    E = (eval_energy(start) + eval_energy(end)) / 2
    P = (eval_power(start) + eval_power(end)) / 2
        
    if power_overflown:
        # endpoints overflown, return infinity
        return (E, P)
    
    x = start + dx
    while (x < end):
        # add energy and power separately
        if not energy_overflown:
            E += eval_energy(x)

        # add power 
        P += eval_power(x)
        if power_overflown: 
            return (E, P)
            
        x += dx
    
    return (E, P)

In [23]:
E = []
samples = 10**5
for i in range(2, 8):
    E.append(compute_E_P(f6, 0, 10**i, 10**i/samples))
E

[(5000.000000000404, 50.00000000000403),
 (500009.99999968, 500.00999999967996),
 (50000000.00004408, 5000.000000004418),
 (5000000000.0, 49999.999999999985),
 (500000000000.0, 499999.99999999994),
 (50000000000000.0, 5000000.0)]